# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096784


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<01:01,  2.20s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:03,  2.34s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:43,  1.68s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:30,  1.23s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  6.65it/s]

Rendering models:  93%|█████████▎| 28/30 [00:10<00:00,  5.87it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  6.26it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  6.57it/s]

Linda_J._Berkel                       0.000219
aliabdulhameed                        0.001682
whitgift                              0.000903
cameronw                              7.505771
Alexander00                           0.000818
Karl_Knopf                            0.000289
ethanth                               0.000203
darkscout                             0.001337
not-logged-in-808d8d7eb7071532ff8d    0.001803
jnarayanbvg                           0.003776
lsautter                              0.000231
not-logged-in-a3896d01d26815cb2abd    0.001253
Jessica_Korkmaz                       0.000857
clars915                              0.000364
Raven_Vasquez                         0.002451
AlexYoung35                           0.000276
gabrielweiss                          0.001522
Tarskii                               0.001607
acapirala                             0.001024
TRISTANM18                            0.000509
dinosora                              0.000231
Lavadude     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())